# Instance Segmentation evaluation

Test a Detector on A Customized Dataset (Mask RCNN)

Notice: MMDetection **only support evaluating mask AP of dataset in COCO format for now**. Other methods and more advanced usages can be found in the [doc](https://mmdetection.readthedocs.io/en/latest/tutorials/customize_dataset.html). So we need to reorganize the dataset into a COCO format firstly.

In [ ]:
# from mmdet.apis import init_detector, inference_detector, show_result_pyplot
from mmdet.apis import init_detector, inference_detector
import mmcv

## Load the model

### Load model architecture

Notice: The .py file (model config file) is generated in the "main_InstanceSeg_train.ipynb"

In [ ]:
# from mmcv import Config
from mmengine.config import Config, DictAction

# load a model architecture
model_file=r'/scratch/tjian/PythonProject/DP_MMDetection/checkpoints/train_weights/Paper_Peng/Yolact_r50/lr_0.001/Pollutant_yolact_r50.py'
cfg = Config.fromfile(model_file)

# print(f'Config:\n{cfg.pretty_text}')

### Load model weights

In [ ]:
# load model weights from checkpoint file (.pth)

# checkpoint_file = r"F:\Tianlong\PythonProject\deep_plastic_MMDetection\checkpoints\train_weights/train1_100_epochs/best_bbox_mAP_epoch_48.pth"
checkpoint_file = r"/scratch/tjian/PythonProject/DP_MMDetection/checkpoints/train_weights/Paper_Peng/Yolact_r50/lr_0.001/best_coco_segm_mAP_50_epoch_43.pth"


In [ ]:
# build the model from a config file and a checkpoint file

# (1) GPU device
model = init_detector(cfg, checkpoint_file, device='cuda:0')

# (2) cpu device
# model = init_detector(cfg, checkpoint_file, device='cpu')

## Evaluation

### (1) Test on a test dataset

In [ ]:
# Single-gpu testing

# "--show-dir" saves the predicted images in test dataset
# "test_evaluator.outfile_prefix": saves the test results in a "pkl" file, this file can be used to predict images

!python tools/test.py \
    checkpoints/train_weights/Paper_Peng/Yolact_r50/lr_0.001/Pollutant_yolact_r50.py \
    /scratch/tjian/PythonProject/DP_MMDetection/checkpoints/train_weights/Paper_Peng/Yolact_r50/lr_0.001/best_coco_segm_mAP_50_epoch_43.pth \
    --work-dir /scratch/tjian/PythonProject/DP_MMDetection/checkpoints/train_weights/Paper_Peng/Yolact_r50/lr_0.001/test/ \
    --out /scratch/tjian/PythonProject/DP_MMDetection/checkpoints/train_weights/Paper_Peng/Yolact_r50/lr_0.001/test/result.pkl
    # --cfg-options \
    # --show-dir /scratch/tjian/PythonProject/DP_MMDetection/checkpoints/train_weights/Paper2_exp5/Oos_20per/FRCNN_Frozen_4_PreTrain_GV/pred_images_in_test_dataset/

### (2) Predict images in a folder

Note: inference_detector only supports single-image inference for now

In [ ]:
# # old version
# import os

# # test images in a folder
# folder_path = r"/scratch/tjian/Data/Pollutant/test/"
# out_path = r"/scratch/tjian/PythonProject/DP_MMDetection/checkpoints/train_weights/Paper_Peng/Yolact_r50/lr_0.001/test/ima_pred/"

# file_names = os.listdir(folder_path)
# for filename in file_names:
#         if filename.endswith('.jpg'):
#             image_path = os.path.join(folder_path, filename)
#             result = inference_detector(model, image_path)
#             print(result)
#             # or save the visualization results to image files
#             out_file_name= os.path.join(out_path, filename)
#             model.show_result(image_path, 
#                               result, 
#                               score_thr=0.5,
#                               out_file=out_file_name)

In [ ]:
import os
from mmdet.apis import init_detector, inference_detector
from mmdet.utils import register_all_modules
from mmdet.registry import VISUALIZERS
import mmcv

config_file =r'/scratch/tjian/PythonProject/DP_MMDetection/checkpoints/train_weights/Paper_Peng/Yolact_r50/lr_0.001/Pollutant_yolact_r50.py'
checkpoint_file = r"/scratch/tjian/PythonProject/DP_MMDetection/checkpoints/train_weights/Paper_Peng/Yolact_r50/lr_0.001/best_coco_segm_mAP_50_epoch_43.pth"

#Register all modules in mmdet into the registries
register_all_modules()
# build the model from a config file and a checkpoint file
model = init_detector(config_file, checkpoint_file, device='cuda:0')  # or device='cpu'


# test images in a folder
folder_path = r"/scratch/tjian/Data/Pollutant/test/"
out_path = r"/scratch/tjian/PythonProject/DP_MMDetection/checkpoints/train_weights/Paper_Peng/Yolact_r50/lr_0.001/test/ima_pred/"

file_names = os.listdir(folder_path)
for filename in file_names:
        if filename.endswith('.jpg'):
            image_path = os.path.join(folder_path, filename)
            img = mmcv.imread(image_path, channel_order='rgb')
            result = inference_detector(model, img)
            # print(result)
            
            # init the visualizer(execute this block only once)
            visualizer = VISUALIZERS.build(model.cfg.visualizer)
            # the dataset_meta is loaded from the checkpoint and
            # then pass to the model in init_detector
            visualizer.dataset_meta = model.dataset_meta
            
            out_file_name= os.path.join(out_path, filename)
            # show the results
            visualizer.add_datasample('result',
                                      img,
                                      data_sample=result,
                                      draw_gt=False,
                                      wait_time=0,
                                      out_file=out_file_name,
                                      pred_score_thr=0.5
                                      )
            

### (3) Confusion matrix

In [ ]:
!python tools/analysis_tools/confusion_matrix.py \
    configs/mask_rcnn/mask_rcnn_r50_caffe_fpn_mstrain-poly_1x_coco_Pollutant.py \
    F:/Tianlong/PythonProject/deep_plastic_MMDetection/checkpoints/train_weights/train1_300_epochs_v4/test/output.pkl \
    F:/Tianlong/PythonProject/deep_plastic_MMDetection/checkpoints/train_weights/train1_300_epochs_v4/test \
    --show \
    --score-thr 0.5 \
    --tp-iou-thr 0.5 
    
    
# ${CONFIG}  ${DETECTION_RESULTS}  ${SAVE_DIR} --show

### (4) Predict images in a folder and save bbox and segm in an excel file

(Only windows system)

Note:

(1) define excel path in untils.bbox_segm.py

(2) define source image folder below

(3) saving results in excel only works on Windows system 

(4) Two folders will be generated in "--out-dir", including (1) predicted images, and (2) bbox and mask info in json file (mask info is encoded)

(5) I mainly modify the "/mmdet/apis/det_inferencer.py" to realize this function, referring: https://blog.csdn.net/m0_46246301/article/details/130561039



In [1]:
!python demo/image_demo.py C:/Users/tjian/Desktop/1/ \
    checkpoints/Pollutant_yolact_r50.py \
    --weights checkpoints/best_coco_segm_mAP_50_epoch_43.pth \
    --out-dir C:/Users/tjian/Desktop/2/ \
    --pred-score-thr 0.5 \
    --device cpu

Loads checkpoint by local backend from path: checkpoints/best_coco_segm_mAP_50_epoch_43.pth
['entrap bean', 'free bean']
09/25 23:31:19 - mmengine - WARNING - Failed to search registry with scope "mmdet" in the "function" registry tree. As a workaround, the current "function" registry in "mmengine" is used to build instance. This may cause unexpected failure when running the built modules. Please check whether "mmdet" is a correct scope, or whether the registry is initialized.
reading image:  C:/Users/tjian/Desktop/1/A10.jpg
reading image:  C:/Users/tjian/Desktop/1/A11.jpg
reading image:  C:/Users/tjian/Desktop/1/A9.jpg
Inference ---------------------------------------- 0.0 it/s  
results have been saved at C:/Users/tjian/Desktop/2/


c:\Users\tjian\Anaconda3\envs\MMDetection\lib\site-packages\mmengine\visualization\visualizer.py:196: UserWarning: Failed to add <class 'mmengine.visualization.vis_backend.LocalVisBackend'>, please provide the `save_dir` argument.
  warnings.warn(f'Failed to add {vis_backend.__class__}, '


In [ ]:
# old version

import os
import pycocotools.mask as mask_util
import utils.bbox_segm as bs
from mmdet.core import encode_mask_results
from utils.polygonFromMask import polygonFromMask

num_classes = 2
classes_labels = ["entrap_bean", "free_bean"]
score_thr = 0.5

# test images in a folder
folder_path = r"U:\AIMMW\Tianlong\Pollutant\dataset\v2\test"
out_path = r"U:\AIMMW\Tianlong\Pollutant\dataset\v2\test_result\test_pred"

# create the "out_path" if they do not exist
if not os.path.isdir(out_path):
    os.mkdir(out_path)

# save results in an excel file
excel_path = r"U:\AIMMW\Tianlong\Pollutant\dataset\v2\test_result\test_pred.xlsx"

filename_list=[]
xmin_list=[]
ymin_list=[]
xmax_list=[]
ymax_list=[]
conf_list=[]
class_bbox_list=[]
area_bbox_list=[]
area_mask_list=[]
# index_list=[]

file_names = os.listdir(folder_path)
for filename in file_names:
    if filename.endswith('.jpg'):
        image_path = os.path.join(folder_path, filename)
        result = inference_detector(model, image_path)
        # print("result:", result)
        # or save the visualization results to image files
        out_file_name= os.path.join(out_path, filename)
        model.show_result(image_path, 
                          result, 
                          score_thr=0.5,
                          out_file=out_file_name)
            
        ## bbox -> result[0] and mask -> result[1] for single image
        encode_result = (result[0], encode_mask_results(result[1])) # containing bbox and segm
        bbox_results, segm_results = encode_result
        # print(bbox_results)

        for i in range(num_classes):
          # print(classes_labels[i], ":")
          
          j=0
          index_list=[]
          
          for bbox_result in bbox_results[i]:
              conf = bbox_result[4]
               
              # only save bbox and segm with the confidence (>score_thr)
              if conf >= score_thr:
                xmin = bbox_result[0]
                ymin = bbox_result[1]
                xmax = bbox_result[2]
                ymax = bbox_result[3]
                conf = bbox_result[4]
                class_bbox = classes_labels[i]
                area_bbox = (xmax-xmin)*(ymax-ymin) 
                index = j
                # print("j: ", j)
              
                filename_list.append(filename)
                xmin_list.append(xmin)
                ymin_list.append(ymin)
                xmax_list.append(xmax)
                ymax_list.append(ymax)
                conf_list.append(conf)
                class_bbox_list.append(class_bbox)
                area_bbox_list.append(area_bbox)
                index_list.append(j)

              j=j+1

          for index_seg in index_list:
            segm_result=segm_results[i][index_seg]
            #  print("##: ",segm_result)
            sample_mask = mask_util.decode(segm_result)
            sample_polygon,_,AreaMask = polygonFromMask(sample_mask)
            
            area_mask_list.append(AreaMask)
            # print("area:", AreaMask)


bs.save_bbox_info_in_excel(excel_path, "Sheet1", filename_list, 
         xmin_list, ymin_list, xmax_list, ymax_list, conf_list, class_bbox_list, area_bbox_list, area_mask_list)